## Importing Packages

In [ ]:
from sqlalchemy import MetaData, create_engine, Table, Column, String, Integer, Float, Boolean,delete
from sqlalchemy import insert, select,func, and_, desc, case, cast

import pandas as pd

## Connecting to Database

In [ ]:
# Create an engine that connects to the census.sqlite file: engine
engine = create_engine('sqlite:///Chinook_Sqlite.sqlite')
# Print table names
print(engine.table_names())

# Defining metadata object
metadata = MetaData()

## Creating Table

In [ ]:
# Creating table if it doesn't exist in database
if not engine.has_table('instrument'):
    data = Table('instrument', metadata,
             Column('InstrumentID', Integer(), unique=True),
             Column('ArtistID', Integer(), unique=True),
             Column('Instrument_Name', String(500)),
             Column('Cost', Integer())
    )
    # Use the metadata to create the table
    metadata.create_all(engine)

## Creating Table Object

In [ ]:
customer = Table('customer', metadata, autoload=True, autoload_with=engine)
instrument = Table('instrument', metadata, autoload=True, autoload_with=engine)
artist = Table('Artist', metadata, autoload=True, autoload_with=engine)

## Getting Table Information

In [ ]:
# Print the column names
print(instrument.columns.keys())
print('---------------------------------')
# Print the table details
print(repr(metadata.tables['instrument']))

## Inserting Data into Table

In [ ]:
## Inserting single row in the table
stmt = insert(instrument).values(InstrumentID=1, ArtistID = 1, Instrument_Name='Guitar', Cost=245)

results = engine.execute(stmt)
# Returns number of rows affected
print(results.rowcount)

In [ ]:
## Inserting multiple row in the table
instrument_list = [
    {'InstrumentID': 2, 'ArtistID': 2, 'Instrument_Name': 'Tabla', 'Cost': 200},
    {'InstrumentID': 3, 'ArtistID': 3, 'Instrument_Name': 'Vilon', 'Cost': 500}
]


stmt = insert(instrument)
results = engine.execute(stmt, instrument_list)

print(results.rowcount)

In [ ]:
# Inserting data from csv file
instruments_csv = pd.read_csv('instruments.csv')

stmt = insert(instrument)

instrument_list = []

for index, row in instruments_csv.iterrows():
    data = {'InstrumentID': row[0], 'ArtistID': row[1], 'Instrument_Name': row[2], 'Cost': row[3]}
    instrument_list.append(data)

results = engine.execute(stmt, instrument_list)
print(results.rowcount)

## Selecting Data

In [ ]:
# Selecting data without using SQLAlchemy package
stmt = 'SELECT * FROM instrument'

results = engine.execute(stmt).fetchall()
print(results)

In [ ]:
stmt = select([instrument])

# SQL Statement while will be executed
print(stmt)

result = engine.execute(stmt).fetchall()

# First row
first_row = result[0]

print(first_row)

# First column in first row
print(first_row[0])

print(first_row['Instrument_Name'])


### Selecting using 'where' Clause

In [ ]:
stmt = select([customer])

stmt = stmt.where(customer.columns.PostalCode == '12227-000')

results = engine.execute(stmt).fetchall()

for result in results:
    print(result)

### Selecting using 'in' Clause

In [ ]:
stmt = select([customer])

stmt = stmt.where(customer.columns.PostalCode.in_(['14700', '12227-000']))

stmt = stmt.order_by(customer.columns.PostalCode)

results = engine.execute(stmt).fetchall()

for result in results:
    print(result)

### Selecting using 'and' Clause

In [ ]:
stmt = select([customer])

stmt = stmt.where(
    and_(
        customer.columns.Country == 'Brazil',
        customer.columns.Email == 'eduardo@woodstock.com.br'
    )
)

results = engine.execute(stmt).fetchall()

for result in results:
    print(result)

### Ordering Data with one Column

In [ ]:
stmt = select([instrument])

stmt = stmt.order_by(instrument.columns.Instrument_Name)

result = engine.execute(stmt).fetchall()
print(result)

### Ordering Data with one Column in Reverse order

In [ ]:
stmt = select([instrument])

stmt = stmt.order_by(desc(instrument.columns.Instrument_Name))

result = engine.execute(stmt).fetchall()
print(result)

### Ordering Data with more than one Column

In [ ]:
stmt = select([instrument])

stmt = stmt.order_by(desc(instrument.columns.Cost), instrument.columns.Instrument_Name)

result = engine.execute(stmt).fetchall()
print(result)

### Aggregating Data

In [ ]:
stmt = select([func.count(instrument.columns.Instrument_Name.distinct())])

distinct_instruments = engine.execute(stmt).scalar()

print(distinct_instruments)

### Grouping up the Data

In [ ]:
stmt = select([instrument.columns.Instrument_Name, 
               func.count(instrument.columns.Instrument_Name), instrument.columns.Cost])

stmt = stmt.group_by(instrument.columns.Instrument_Name)

results = engine.execute(stmt).fetchall()

print(results)

### Renaming Column

In [ ]:
instruments_count = func.count(instrument.columns.Instrument_Name).label('instruments_count')

stmt = select([instruments_count, instrument.columns.Instrument_Name, instrument.columns.Cost])

stmt = stmt.group_by(instrument.columns.Instrument_Name)

results = engine.execute(stmt).fetchall()

print(results)

print(results[0].keys())

### Case and Cast Statements

In [ ]:
price_gt_250 = case([
        (instrument.columns.Cost < 250, True)
    ], else_= False)

total_pop2000 = cast(price_gt_250, Boolean)

stmt = select([instrument.columns.Instrument_Name, price_gt_250])

result = engine.execute(stmt).fetchall()
# Print the percentage
print(result)

### Arithmetic Operations

In [ ]:
stmt = select([customer.columns.CustomerId, (customer.columns.FirstName + ' ' + customer.columns.LastName)
           .label('customer_name')])

stmt = stmt.order_by('customer_name')

stmt = stmt.limit(5)
print(stmt)
results = engine.execute(stmt).fetchall()

for result in results:
    print(result, ' ')

## Joining Tables

In [ ]:
stmt = select([instrument, artist])

stmt = stmt.select_from(
    instrument.join(artist, instrument.columns.ArtistID == artist.columns.ArtistId))
print(stmt)
result = engine.execute(stmt).fetchall()

# Printing first two items from result set
print(result[:2])

## Updating Data

In [ ]:
stmt = update(instrument).values(Cost = 200).where(instrument.columns.Cost < 250)

results = engine.execute(stmt)

print(results.rowcount)

# checking data after updating it
stmt = select([instrument])
results = engine.execute(stmt).fetchall()
print(results)

## Creating Pandas Dataframe out of Result Set

In [ ]:
stmt = select([instrument])
results = engine.execute(stmt).fetchall()

df = pd.DataFrame(results)

df.columns = results[0].keys()

print(df)

## Deleting Data based on a Condition

In [ ]:
stmt = delete(instrument)
stmt = stmt.where(instrument.columns.Cost < 250)

results = engine.execute(stmt)

print(results.rowcount)

## Deleting all of the Data

In [ ]:
stmt = delete(instrument)

results = engine.execute(stmt)

print(results.rowcount)

## Deleting Table
Be very sure before running below commands as it will delete table from database

In [ ]:
instrument.drop(engine)
print(instrument.exists(engine))

# Dropping all tables
metadata.drop_all(engine)
print(customer.exists(engine))